<a href="https://colab.research.google.com/github/ClarkYang91/COMP90042_Project_Automatic_Fact_Verification/blob/master/run_online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.37.47.154:8470

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15449808202064838863),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10706889175475127489),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17747674755982015346),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2913308007495403465),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10754104947629579678),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5507647987916691267),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 1

In [4]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 234.66 KiB | 3.21 MiB/s, done.
Resolving deltas: 100% (186/186), done.


W0520 03:32:42.540789 140459742041984 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [5]:
BUCKET = 'group_project' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models'.format(BUCKET)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


BERT_MODEL = 'uncased_L-24_H-1024_A-16' #@param {type:"string"}
# TODO bert_hub_module option 'bert_uncased_L-24_H-1024_A-16/1'
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Model output directory: gs://group_project/bert-tfhub/models *****


In [6]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)

Instructions for updating:
Colocations handled automatically by placer.


W0520 03:33:14.552318 140459742041984 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0520 03:33:19.119438 140459742041984 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
import pickle

TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
          guid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

class DataProcessor(object):

    def __init__(self):
        self.dummy = None

    def get_labels(self):
        return ['SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO']

    def get_train_examples(self):
        with open('train30000.txt', 'rb') as f:
            df = pickle.load(f)
        input_examples = df.apply(lambda x: InputExample(guid=x['index'],
                                                             text_a=x['claim'],
                                                             text_b=x['evidence'],
                                                             label=x['label']), axis=1)
        return input_examples
        

    def get_dev_examples(self):
        with open('devinf.txt', 'rb') as f:
            df = pickle.load(f)
        input_examples = df.apply(lambda x: InputExample(guid=x['index'],
                                                             text_a=x['claim'],
                                                             text_b=x['evidence'],
                                                             label=x['label']), axis=1)
        return input_examples

    def get_test_examples(self):
        with open('test.txt', 'rb') as f:
            df = pickle.load(f)
        input_examples = df.apply(lambda x: InputExample(guid=x['index'],
                                                             text_a=x['claim'],
                                                             text_b=x['evidence'],
                                                             label='NOT ENOUGH INFO'), axis=1)
        return input_examples


processor = DataProcessor()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples()
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))



In [13]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


W0520 03:56:42.540772 140459742041984 estimator.py:1924] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fbf08bd6f28>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_model_dir': 'gs://group_project/bert-tfhub/models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.37.47.154:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf07a5d080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.37.47.154:8470', '_evaluation_master': 'grpc://10.37.47.154:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_

I0520 03:56:42.548506 140459742041984 estimator.py:201] Using config: {'_model_dir': 'gs://group_project/bert-tfhub/models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.37.47.154:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf07a5d080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.37.47.154:8470', '_evaluation_master': 'grpc://10.37.47.154:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0520 03:56:42.553287 140459742041984 tpu_context.py:202] _TPUContext: eval_on_tpu True


In [0]:
def model_train(estimator):
  print('Start training')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [44]:
model_train(estimator_from_tfhub)

Start training
INFO:tensorflow:Writing example 0 of 90000


I0519 14:48:01.241928 140037330216832 run_classifier.py:774] Writing example 0 of 90000


INFO:tensorflow:*** Example ***


I0519 14:48:01.246548 140037330216832 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 53454


...

INFO:tensorflow:Loss for final step: 3.6000863e-05.


I0519 16:32:19.019183 140037330216832 estimator.py:359] Loss for final step: 3.6000863e-05.


INFO:tensorflow:training_loop marked as finished


I0519 16:32:19.022475 140037330216832 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-05-19 16:32:19.028254 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples()
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [14]:
model_eval(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 7354


I0520 03:56:49.838046 140459742041984 run_classifier.py:774] Writing example 0 of 7354


INFO:tensorflow:*** Example ***


I0520 03:56:49.841874 140459742041984 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 4002


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 33750: gs://group_project/bert-tfhub/models/model.ckpt-33750


I0520 03:58:10.911247 140459742041984 estimator.py:2039] Saving 'checkpoint_path' summary for global step 33750: gs://group_project/bert-tfhub/models/model.ckpt-33750


INFO:tensorflow:evaluation_loop marked as finished


I0520 03:58:11.467761 140459742041984 error_handling.py:93] evaluation_loop marked as finished


***** Finished evaluation at 2019-05-20 03:58:11.473342 *****
***** Eval results *****
  eval_accuracy = 0.8688792
  eval_loss = 1.0983539
  global_step = 33750
  loss = 0.9595884
